# this is the Capstone project for IBM Data Sicence Professional Certificate

In [1]:
#import library
import pandas as pd
import numpy as np

In [2]:
# start as request
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [7]:
#import module for scraping
from bs4 import BeautifulSoup
import lxml
import html5lib
import requests
print('done')

done


#get html from web
#get the table and rows to parse
#create the dataframe to store the result from loop

In [11]:

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [42]:

table = soup.table
table_rows=table.find_all('tr')

In [171]:

df = {'postcode':[],'Borough':[],'Neighbourhood':[]}

Neigh = pd.DataFrame(df)

type(Neigh)

pandas.core.frame.DataFrame

#create the list to append to the dataframe
#loop to assign value to the list
#reference link: https://www.youtube.com/watch?v=sAuGH1Kto2I&list=PL0vuEoxnycCKWtKG-cOh1JcIwNNraQ8sP&index=4&t=0s

# the result from loop, the first result set is an empyty list as header and without index, need the try block to exclude that empty list, otherwise error
# reference link: https://www.youtube.com/watch?v=ng2o98k983k

In [172]:

l=[]


for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
   
    try:
        l=[{'postcode':row[0],'Borough':row[1],'Neighbourhood':row[2]}]
        Neigh=Neigh.append(l,sort=True)
    except Exception as e:
        pass

In [173]:
Neigh.shape

(180, 3)

In [196]:
# original dataframe has all row index as 0, reset index
Neigh.reset_index(drop=True)
# reset column order
N = Neigh[['postcode','Borough','Neighbourhood']]
N = Neighbourhood.reset_index(drop=True)

In [199]:
# Drop borough that are not assigned
New = N[N.Borough != 'Not assigned\n']
#Remove all '\n', and replace '/' with ','
R = New.replace('\n','',regex=True)
dataframe = R.replace('/',',',regex=True)


In [202]:
dataframe.head()

,postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [203]:
dataframe.shape

(103, 3)